# RF Complexity Analysis (EU)
## Data Loading

We use the `forest_report.json` file in the repo. Each entry contains the dataset name, metadata (sizes, series length, split), and trained forest statistics.

In [ ]:
from etl.tables import (
    prepare_models_analysis,
    print_models_analysis_overview,
    print_models_analysis_diagnostics
)

analysis_context = prepare_models_analysis(verbose=True)
print_models_analysis_overview(analysis_context)

## Summary Table

We build a table with: `dataset`, `analyzed` (if results exist in `results/`), dataset sizes (`train_size`, `test_size`, `series_length`), model size (`n_estimators`), and EU stats (`mean eu features`, `eu std`).

Correct sorting (priority):
- `n_estimators` ascending
- `eu_complexity` ascending (fallback to `series_length` when missing)
- `series_length` ascending
- `dataset` alphabetical


In [ ]:
# Display first table (summary with dataset info and EU stats)
analysis_context.first_table.summary_styler

## Analyzed Datasets Count Table

Shows detailed statistics for datasets that have been analyzed (have results in `results/` directory).

In [ ]:
# Display analyzed counts table
analysis_context.analyzed_counts_styler

## Combined Analysis Table

Combined view of dataset metadata and analysis results.

In [ ]:
# Display combined table
analysis_context.combined_analyzed_styler

## Redis Summary

Summary of Redis backup analysis showing reason counts by category.

In [ ]:
# Display redis summary with diagnostics
from etl.tables import print_models_analysis_diagnostics

print_models_analysis_diagnostics(analysis_context)
